# Mapper of bike stations/bus stops

In [2]:
#before running the script, be sure you have geopy and folium. Use pip install to get them.

In [3]:
import pandas as pd
from geopy.distance import vincenty

# read Soren's "toBike_stations_only.csv" file with pandas

## attention!!! Soren's file carries non-ascii characters (e.g. "`"). Check the file and correct it

In [4]:
bike = pd.read_csv("../stats_zones/toBike_stations_and_test_stations.csv", sep=',')

In [5]:
bike = bike.sort(('lat', 'long'))

In [6]:
bike.head()

,id,name,lat,long,total_station_size,zone
104,105,Pio VII,45.024985,7.652120,12,1
105,106,Pio VII 2,45.029465,7.654760,16,1
68,69,Lingotto Fiere,45.031050,7.666585,22,1
155,156,Ventimiglia,45.032740,7.671595,15,1
53,54,Giambone,45.033055,7.647035,17,1


# read "bus_in_torino.csv" file with pandas

In [7]:
bus = pd.read_csv("../bus_stops/torino_it/bus_in_torino.csv")

In [8]:
bus = bus.sort(('stop_lat', 'stop_lon'))

In [9]:
bus.head()

,Unnamed: 0,stop_id,stop_lat,stop_lon
2159,4485,53696001,45.00420,7.69030
2605,6794,57322001,45.00422,7.65981
2681,6956,53696002,45.00423,7.69011
1801,2785,52825001,45.00423,7.72853
1862,3112,53640003,45.00426,7.66144


# Build a dictionary with the station name and the bus stops within 350m

In [10]:
D = {}
c = 0
maxDistance = 350

In [11]:
for i,j in bike.iterrows():
    temp = []
    distances = []
    for k,h in bus.iterrows():
        #counter to check if the function is working, it should finish with 444675 comparisons
        if c % 100000 == 0 and c != 0:
            print c

        distance = vincenty([j[2], j[3]], [h[2], h[3]]).meters
    
        if distance < maxDistance:
            temp.append(int(h[1]))
            distances.append(distance)
        
        c += 1
        
    if temp:
        av = reduce(lambda x, y: x + y, distances) / len(distances)
        D[j[1]] = ({'id': [j[0]], 'coord': [j[2], j[3]], 'stops':temp, 'stops#':len(temp), 'average':av})

print 'finished in', c, 'comparisons' 

100000
200000
300000
400000
finished in 460845 comparisons


In [12]:
D

{'Adriano': {'average': 218.08115580565646,
  'coord': [45.070665000000005, 7.65597],
  'id': [1],
  'stops': [54494002,
   51551007,
   51720010,
   51720005,
   51573007,
   51720006,
   51720009,
   51608008,
   51608007,
   52511004,
   52511005,
   51562004,
   51557008,
   51614007,
   51614006,
   54729001],
  'stops#': 16},
 'Arbarello': {'average': 240.29128697569288,
  'coord': [45.07356, 7.676585],
  'id': [2],
  'stops': [57271001,
   52081006,
   52081007,
   52056005,
   52177002,
   52066002,
   52091003,
   52076002,
   52087003,
   52115003,
   55564001,
   55565001,
   57127001,
   55562001,
   55563001,
   52124004],
  'stops#': 16},
 'Arcivescovado': {'average': 244.56466922271412,
  'coord': [45.06706, 7.678175],
  'id': [3],
  'stops': [52137003,
   52116002,
   56157003,
   57878001,
   57878002,
   52051003,
   52063002,
   52094002,
   51851006,
   52078004,
   52164002,
   52106002,
   56682001,
   52113003,
   52113005,
   57192001,
   52113004,
   52138002,


In [13]:
with open('bike_bus_testStations.csv', 'wb') as f:
    arr = D.keys()
    f.write('id;name;mall#;average;malls;coord\n')
    for i,j in enumerate(D.values()):
        name = arr[i]
        string = ''
        for i in xrange(len(j.values())):
            t = str(j.values()[i])

            if '[' in t:
                t = t.replace('[', '')
            if ']' in t:
                t = t.replace(']', '')
            if "'" in t:
                t = t.replace("'", "")
                
            string += t + ';'
        
        f.write(name+';'+string+'\n')

# let's make the map

In [12]:
import folium

In [13]:
map_1 = folium.Map(location=[45.06, 7.65], zoom_start=12)

for i,j in D.iteritems():
#    print D_pos["Piazza d'Armi"]
    folium.RegularPolygonMarker(j['coord'], color='red', fill_color='red', number_of_sides=4, radius=3).add_to(map_1)
    folium.CircleMarker(j['coord'], popup=i+'\t'+str(j['stops#'])+' bus stops at '+str(int(j['average']))+'m on average',
                        radius=int(j['stops#'])*10, color='#3186cc', fill_color='#3186cc').add_to(map_1)

map_1.save('bikeBus1.html')